# Organização de Vídeos ICT 2019-2020

## Inicialização

In [55]:
# Incluindo diretório scripts no caminho
from glob import glob
import sys
import os
sys.path.insert(0, './DATA/')

In [56]:
# show result from all calculations of the cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [57]:
# pacotes básicos
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('max_colwidth',250)
import unidecode
import re

## Carregando dados

In [58]:
colunas = ['ID', 'Título do Projeto', 'Docente', 'Centro', 'Unidade', 
           'Área Conhecimento', 'Aluno', 'CPF Aluno', 'Cota Bolsa','TIPO']

In [59]:
servidores = pd.read_excel('./DATA/UFCG_Servidores_Ativos.xls', converters={'CPF':str})
servidores = servidores[['CPF','Nome']]


formularios = pd.read_csv('./DATA/Formulário_ICT_VIDEOS.csv', converters={'CPF_ALUNO':str, 'CPF_ORIENTADOR':str})
formularios = formularios.dropna(axis=1, how='any')
formularios['CPF_ORIENTADOR'] = formularios['CPF_ORIENTADOR'].apply(
    lambda row: re.sub(r"\D", "", row))
formularios['CPF_ALUNO'] = formularios['CPF_ALUNO'].apply(
    lambda row: re.sub(r"\D", "", row))
formularios['TIMESTAMP'] = pd.to_datetime(formularios['TIMESTAMP'])

PIBIC = pd.read_excel('./DATA/PIBIC-2019.xls', converters={'CPF Aluno':str})
PIVIC = pd.read_excel('./DATA/PIVIC-2019.xls', converters={'CPF Aluno':str})
PIVIC_1 = pd.read_excel('./DATA/PIVIC-2019-1.xls', converters={'CPF Aluno':str})
PIBITI = pd.read_excel('./DATA/PIBITI-2019.xls', converters={'CPF Aluno':str})
PIVITI = pd.read_excel('./DATA/PIVITI-2019.xls', converters={'CPF Aluno':str})

PIBIC['TIPO'] = 'PIBIC' 
PIVIC['TIPO'] = 'PIVIC' 
PIVIC_1['TIPO'] = 'PIVIC' 
PIBITI['TIPO'] = 'PIBITI' 
PIVITI['TIPO'] = 'PIVITI' 

projetos = pd.concat([PIBIC,PIVIC,PIVIC_1, PIBITI, PIVITI])
projetos = projetos[colunas]
projetos = projetos.rename(columns={'Título do Projeto':'TITULO',
                                    'Docente':'DOCENTE',
                                    'Centro':'CENTRO',
                                    'Unidade':'UA',
                                    'Área Conhecimento':'AREA',
                                    'Aluno':'ALUNO',
                                    'CPF Aluno':'CPF_ALUNO',
                                    'Cota Bolsa':'BOLSA'})
projetos.CPF_ALUNO = projetos.CPF_ALUNO.astype(str)
projetos['DOCENTE'] = projetos['DOCENTE'].apply(
    lambda row: unidecode.unidecode(row))

dados_projetos = pd.merge(
    left=projetos, right=servidores,
    left_on='DOCENTE', right_on='Nome').drop('Nome',1)

dados_videos = pd.merge(
    left=dados_projetos, right=formularios,
    left_on='CPF_ALUNO', right_on='CPF_ALUNO')

videos = dados_videos[
    dados_videos['CPF_ORIENTADOR'] == dados_videos['CPF']
]
#Eliminando Duplicatas e ficando com a versão mais recente.
videos_validos = videos.sort_values('TIMESTAMP').drop_duplicates('CPF_ALUNO',keep='last')

#videos_validos['LINK_VIDEO']=videos_validos.apply(lambda row: row['LINK_VIDEO'].split['='][-1])


videos_validos['title'] = videos_validos.apply(lambda row: 
                     'ICT-2020-' + row['AREA'] + '-' + '-' + row['CENTRO'] +
                     '-' + row['UA'] + ':  ' + 
                     str(row['TITULO'])[0 : min(len(str(row['TITULO'])),30)] + '...',
                     axis = 1
                    )

videos_validos['description'] = 'Videos para avaliação dos projetos de Iniciação Científica e Tecnológica, vigência 2019 - 2020, da Universidade Federal de Campina Grande. Provisoriamente armazenados nesta canal por problemas técnicos.'

videos_validos['category'] = '27'

videos_validos['keywords'] = videos_validos.apply(lambda row: 
                     'UFCG,PRPG,PIBIC,PIBITI,' + row['AREA'] +',' + row['CENTRO'],
                     axis = 1
                    )

videos_validos['categorias'] = videos_validos.apply(lambda row: row['CENTRO']+'-'+row['UA'], axis = 1)

In [60]:
videos_validos.head(1)['UA'].iloc[0]

'UAMV'

## Criando Listas

In [61]:
listas = ['CENTRO','UA','AREA']
categorias = [videos_validos[el].unique().tolist() for el in listas]
playlists = [ videos_validos[ videos_validos[el] == categ ]['ID'] 
 for el in listas for categ in categorias[listas.index(el)] ]


In [62]:
column_names = ['TIPO', 'CRITERIO', 'DADOS']
df = pd.DataFrame(columns = column_names)
POS = 0
for item in listas:
    for item2 in categorias[listas.index(item)]:
        df.loc[len(df)] = [item, item2, videos_validos[videos_validos.index.isin(playlists[POS].index)]]
        POS += 1        

## Teste

In [75]:
df1 = df[df['TIPO']=='UA']
df1[df1['CRITERIO']=='UAF'].iloc[0,2]['LINK_VIDEO'].to_list()

['https://drive.google.com/u/0/open?usp=forms_web&id=1MPdpQtkQX7DSimBzgYYpSEm02ngODSQI',
 'https://drive.google.com/u/0/open?usp=forms_web&id=1Ehg__XJTKFx3MP9rnYadSvfo05qz-OtF',
 'https://drive.google.com/u/0/open?usp=forms_web&id=1H9mUF4I0NId4wyEefxXTMhUD3l9yRHdk']

In [66]:
uamv.columns

Index(['ID', 'TITULO', 'DOCENTE', 'CENTRO', 'UA', 'AREA', 'ALUNO', 'CPF_ALUNO',
       'BOLSA', 'TIPO', 'CPF', 'TIMESTAMP', 'EMAIL', 'NOME', 'CPF_ORIENTADOR',
       'LINK_VIDEO', 'title', 'description', 'category', 'keywords'],
      dtype='object')

In [67]:
uamv = uamv[uamv['ID']==7733]

In [75]:
uamv.values[0,-5].split('=')[-1]

'16ubBh2OqK_dVFdRQf0rWw_49WMq2KxZp'

In [25]:
a = 1
a = b

NameError: name 'b' is not defined